In [ ]:
!pip install gradio
import openai
import gradio as gr

openai.api_key = "REPLACE WITH KEY"


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["Wordware.pdf"]).load_data()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [ ]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [ ]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to Wordware"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific answers about Wordware."
    ),
)

In [ ]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [ ]:
response = query_engine.query("What is Wordware and its capabilities?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to Wordware.
Wordware is an IDE for building AI agents with natural language programming. It offers a collaborative environment for creating AI agents and features an innovative programming language that combines plain English with essential programming concepts. Wordware supports forking, API deployment, and hosted app sharing, making it easy to collaborate and deploy AI solutions. Additionally, it allows users to add inputs to agents, utilize a variety of language models from providers like OpenAI, Anthropic, and Claude, connect to a code editor, and create advanced workflows using conditional branching.


In [ ]:
print(len(response.source_nodes))

1


In [ ]:
response = query_engine.query(
    "How do I create an AI agent that connects to the gpt LLM?"
)
print(str(response))

Selecting query engine 1: Creating an AI agent that connects to the gpt LLM requires retrieving specific answers about Wordware..
To create an AI agent that connects to the GPT LLM in Wordware, you should start by building the agent and then use the prompt section to invoke the LLM. Begin by creating a detailed prompt, then type "/" to bring up a list of options. From there, you can select the GPT LLM from providers like OpenAI, Anthropic, Claude, and more. Remember that in Wordware, the LLM executes prompts before it is invoked, so ensure you have a well-crafted prompt before invoking the LLM.


In [ ]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("Wordware.pdf")

In [ ]:
response = query_engine.query("Tell me about how to pass variables from inputs to other sections in the agent?")
print(str(response))

Selecting query engine 1: This choice is more relevant as it focuses on retrieving specific context from the MetaGPT paper, which may contain information on passing variables from inputs to other sections in the agent..
To pass variables from inputs to other sections in the agent, you can use the '@' symbol. This symbol allows you to use the inputs as variables that can be passed around to the code editor, LLM generations, or any other feature within the environment.


In [57]:
def chatbot():
    print("Wordware IDE Chat. Ask any questions about Wordware and the Wordware IDE. Type 'exit' to end the conversation.")
    history = []

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Ending conversation.")
            break

        # Create the full conversation history string
        conversation_history = "\n".join(f"You: {line['user']}\nBot: {line['bot']}" for line in history)
        conversation_with_input = f"{conversation_history}\nYou: {user_input}\n"

        # Get the response from the query engine
        response = query_engine.query(user_input)
        response_text = str(response)

        # Print the response
        print(f"Bot: {response_text}")

        # Update the history
        history.append({"user": user_input, "bot": response_text})

# Start the chatbot
chatbot()

Chatbot initialized. Type 'exit' to end the conversation.
You: What is Wordware
Selecting query engine 1: Wordware is a specific term related to the MetaGPT paper, therefore choice 2 is more relevant for retrieving specific context..
Bot: Wordware is an IDE designed for building AI agents with natural language programming. It includes key components such as a collaborative environment for creating AI agents, an innovative programming language that combines plain English with essential programming concepts, support for forking, API deployment, and hosted app sharing, and a platform tailored for AI agents with a proprietary programming language at its foundation.
You: exit
Ending conversation.
